In [3]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import gemini



url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '7d4a50aa-3354-41e4-ac2d-9e1d6dd1a873',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)
  

ModuleNotFoundError: No module named 'gemini'

In [13]:
import dash
from dash import dcc, html
import pandas as pd
from dash.dependencies import Input, Output
import requests
from collections import deque

# Function to fetch cryptocurrency data
def fetch_crypto_data():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
        'start': '1',
        'limit': '100',  # Fetching top 100 cryptocurrencies
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '7d4a50aa-3354-41e4-ac2d-9e1d6dd1a873',  # Replace with your actual API key 
    }

    session = requests.Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data["data"])
        df["price"] = df["quote"].apply(lambda x: x["USD"]["price"])
        df["market_cap"] = df["quote"].apply(lambda x: x["USD"]["market_cap"])
        df["volume_24h"] = df["quote"].apply(lambda x: x["USD"]["volume_24h"])
        df["percent_change_24h"] = df["quote"].apply(lambda x: x["USD"]["percent_change_24h"])
        df["circulating_supply"] = df["circulating_supply"]
        return df[["name", "price", "market_cap", "volume_24h", "percent_change_24h", "circulating_supply"]]
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return pd.DataFrame(columns=["name", "price", "market_cap", "volume_24h", "percent_change_24h", "circulating_supply"])

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Cryptocurrency Price Tracker"

# External CSS
app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})

# Store historical prices, highest prices, and lowest prices for all cryptocurrencies
historical_prices = {}
highest_prices = {}
lowest_prices = {}

# Layout with a dropdown for currency selection and KPI display
app.layout = html.Div([
    html.H1("Cryptocurrency Price Tracker", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='crypto-dropdown',
        options=[],
        value='Bitcoin',  # Default value
        multi=False,
        clearable=False,
        style={'width': '50%', 'margin': 'auto'}
    ),
    html.Div(id='kpi-display', style={'margin': '20px'}),
    dcc.Graph(id='price-tracker-graph'),
    dcc.Interval(id='interval-component', interval=1*60*1000, n_intervals=0),  # Refresh every minute
], style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'})

# Callback to update the dropdown options with available cryptocurrencies
@app.callback(
    Output('crypto-dropdown', 'options'),
    Input('interval-component', 'n_intervals')
)
def update_dropdown(n):
    df = fetch_crypto_data()
    options = [{'label': row['name'], 'value': row['name']} for index, row in df.iterrows()]
    print("Dropdown options:", options)  # Debugging line
    return options

# Callback to update historical prices for all cryptocurrencies
@app.callback(
    Output('price-tracker-graph', 'figure'),
    Input('crypto-dropdown', 'value'),
    Input('interval-component', 'n_intervals')
)
def update_graph(selected_crypto, n):
    df = fetch_crypto_data()
    print("Selected cryptocurrency:", selected_crypto)  # Debugging line
    print("DataFrame contents:", df)  # Debugging line

    # Update historical prices, highest prices, and lowest prices for all cryptocurrencies
    for index, row in df.iterrows():
        crypto_name = row['name']
        current_price = row['price']
        
        # Initialize historical_prices, highest_prices, and lowest_prices for new cryptocurrencies
        if crypto_name not in historical_prices:
            historical_prices[crypto_name] = deque(maxlen=60)
            highest_prices[crypto_name] = current_price
            lowest_prices[crypto_name] = current_price
        
        # Store the latest price
        historical_prices[crypto_name].append((n, current_price))  # Store (time, price)
        
        # Update the highest price
        if current_price > highest_prices[crypto_name]:
            highest_prices[crypto_name] = current_price
        
        # Update the lowest price
        if current_price < lowest_prices[crypto_name]:
            lowest_prices[crypto_name] = current_price

    # Check if the selected cryptocurrency exists in historical_prices
    if selected_crypto not in historical_prices:
        return {
            'data': [],
            'layout': {
                'title': 'No data available',
                'xaxis': {'title': 'Time (minutes)'},
                'yaxis': {'title': 'Price (USD)'},
            }
        }

    # Prepare data for the selected cryptocurrency's graph
    x_data = [time for time, price in historical_prices[selected_crypto]]
    y_data = [price for time, price in historical_prices[selected_crypto]]

    # Create a figure for the selected cryptocurrency's price
    price_fig = {
        'data': [
            {'x': x_data, 'y': y_data, 'type': 'line', 'name': selected_crypto},
        ],
        'layout': {
            'title': f'{selected_crypto} Price Tracker',
            'xaxis': {'title': 'Time (minutes)'},
            'yaxis': {'title': 'Price (USD)'},
            'plot_bgcolor': '#f9f9f9',
            'paper_bgcolor': '#f9f9f9',
            'font': {'color': '#333'}
        }
    }

    return price_fig

# Callback to update the KPI display
@app.callback(
    Output('kpi-display', 'children'),
    Input('crypto-dropdown', 'value'),
    Input('interval-component', 'n_intervals')
)
def update_kpi(selected_crypto, n):
    df = fetch_crypto_data()
    print("Selected cryptocurrency for KPI:", selected_crypto)  # Debugging line
    print("DataFrame contents for KPI:", df)  # Debugging line
    
    # Ensure the selected cryptocurrency is available in the DataFrame
    if selected_crypto not in df['name'].values:
        return html.Div([html.H4("Cryptocurrency not found.")])

    selected_data = df[df['name'] == selected_crypto].iloc[0]
    highest_price = highest_prices.get(selected_crypto, "N/A")
    lowest_price = lowest_prices.get(selected_crypto, "N/A")

    return html.Div([
        html.H4(f"{selected_crypto} KPIs:", style={'textAlign': 'center'}),
        html.P(f"Current Price: ${selected_data['price']:.2f}", style={'textAlign': 'center'}),
        html.P(f"Market Cap: ${selected_data['market_cap']:.2f}", style={'textAlign': 'center'}),
        html.P(f"24h Trading Volume: ${selected_data['volume_24h']:.2f}", style={'textAlign': 'center'}),
        html.P(f"24h Price Change: {selected_data['percent_change_24h']:.2f}%", style={'textAlign': 'center'}),
        html.P(f"Circulating Supply: {selected_data['circulating_supply']:.2f}", style={'textAlign': 'center'}),
        html.P(f"Highest Ever Price: ${highest_price:.2f}", style={'textAlign': 'center'}),
        html.P(f"Lowest Ever Price: ${lowest_price:.2f}", style={'textAlign': 'center'}),
    ], style={'backgroundColor': '#f9f9f9', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'})

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

c:\Users\ignac\Python\env\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



Dropdown options: [{'label': 'Bitcoin', 'value': 'Bitcoin'}, {'label': 'Ethereum', 'value': 'Ethereum'}, {'label': 'Tether USDt', 'value': 'Tether USDt'}, {'label': 'XRP', 'value': 'XRP'}, {'label': 'BNB', 'value': 'BNB'}, {'label': 'Solana', 'value': 'Solana'}, {'label': 'USDC', 'value': 'USDC'}, {'label': 'Dogecoin', 'value': 'Dogecoin'}, {'label': 'Cardano', 'value': 'Cardano'}, {'label': 'TRON', 'value': 'TRON'}, {'label': 'Chainlink', 'value': 'Chainlink'}, {'label': 'Litecoin', 'value': 'Litecoin'}, {'label': 'Avalanche', 'value': 'Avalanche'}, {'label': 'Sui', 'value': 'Sui'}, {'label': 'Stellar', 'value': 'Stellar'}, {'label': 'Toncoin', 'value': 'Toncoin'}, {'label': 'Shiba Inu', 'value': 'Shiba Inu'}, {'label': 'UNUS SED LEO', 'value': 'UNUS SED LEO'}, {'label': 'Hedera', 'value': 'Hedera'}, {'label': 'Polkadot', 'value': 'Polkadot'}, {'label': 'MANTRA', 'value': 'MANTRA'}, {'label': 'Hyperliquid', 'value': 'Hyperliquid'}, {'label': 'Bitcoin Cash', 'value': 'Bitcoin Cash'}, {

c:\Users\ignac\Python\env\Lib\site-packages\dash\resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



Selected cryptocurrency: Bitcoin
DataFrame contents:                name         price    market_cap    volume_24h  \
0           Bitcoin  85751.722986  1.700434e+12  7.017163e+10   
1          Ethereum   2336.648335  2.817491e+11  3.117109e+10   
2       Tether USDt      0.998962  1.421569e+11  1.262319e+11   
3               XRP      2.213118  1.281126e+11  6.823299e+09   
4               BNB    615.150570  8.764435e+10  2.075187e+09   
..              ...           ...           ...           ...   
95            Ronin      0.986162  6.108183e+08  2.482951e+07   
96         PAX Gold   2906.673098  6.079587e+08  3.530213e+07   
97       MultiversX     21.680064  6.052476e+08  3.228820e+07   
98  Curve DAO Token      0.468618  6.038298e+08  1.294415e+08   
99        dogwifhat      0.594154  5.934655e+08  3.766841e+08   

    percent_change_24h  circulating_supply  
0            -3.173797        1.982973e+07  
1            -5.724784        1.205783e+08  
2             0.004513        1

> No, I cannot access your local files.  I am a large language model; I operate within a contained environment and have no access to your computer or any files stored on it.  I only receive and process the text you provide directly in our current conversation.
